In [1]:
using DataFrames
using CSV
using Plots
using Random
using DecisionTree

In [49]:
#data=CSV.read("train.csv",DataFrame) 
data=CSV.read("balanced_data.csv",DataFrame)
println("truenum:",sum(data[:,end].==true))
println("falsenum:",sum(data[:,end].==false))




function split_train_test(df, train_ratio)
    # 获取数据集的行数
    num_rows = nrow(df)
    
    # 随机排列数据集的行索引
    indices = shuffle(1:num_rows)
    
    # 计算训练集和测试集的大小
    train_size = Int(floor(train_ratio * num_rows))
    
    # 获取训练集和测试集的索引
    train_indices = indices[1:train_size]
    test_indices = indices[train_size+1:end]
    
    # 根据索引划分数据集
    train_set = df[train_indices, :]
    test_set = df[test_indices, :]
    
    return train_set, test_set
end

train,test=split_train_test(data,0.8)

println(train[1:20,:])

println(test[1:20,:])

truenum:23064
falsenum:23064


20×23 DataFrame
 Row │ id     loc      v(g)     ev(g)    iv(g)    n        v        l        d        i        e              b        t        lOCode  lOComment  lOBlank  locCodeAndComment  uniq_Op  uniq_Opnd  total_Op  total_Opnd  branchCount  defects 
     │ Int64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64        Float64  Float64  Int64   Int64      Int64    Int64              Float64  Float64    Float64   Float64     Float64      Bool    
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 80595    122.0     28.0     13.0     12.0    427.0  2546.32     0.03    39.59    56.48      1.02259e5     0.85  5681.04      86          7       18                  0     25.0       46.0     275.0       150.0         55.0     true
   2 │ 83343     65.0      2.

In [3]:
missing_counts = [sum(ismissing.(data[!, col])) for col in names(data)]

# 将结果与列名对应起来
missing_data = DataFrame(Column = names(data), Missing_Count = missing_counts)

println(missing_data)


23×2 DataFrame
 Row │ Column             Missing_Count 
     │ String             Int64         
─────┼──────────────────────────────────
   1 │ id                             0
   2 │ loc                            0
   3 │ v(g)                           0
   4 │ ev(g)                          0
   5 │ iv(g)                          0
   6 │ n                              0
   7 │ v                              0
   8 │ l                              0
   9 │ d                              0
  10 │ i                              0
  11 │ e                              0
  12 │ b                              0
  13 │ t                              0
  14 │ lOCode                         0
  15 │ lOComment                      0
  16 │ lOBlank                        0
  17 │ locCodeAndComment              0
  18 │ uniq_Op                        0
  19 │ uniq_Opnd                      0
  20 │ total_Op                       0
  21 │ total_Opnd                     0
  22 │ branchCount    

In [50]:
train=train[:,2:end]
test=test[:,2:end]

println(train[1:20,:])

println(test[1:20,:])

20×22 DataFrame
 Row │ loc      v(g)     ev(g)    iv(g)    n        v        l        d        i        e              b        t        lOCode  lOComment  lOBlank  locCodeAndComment  uniq_Op  uniq_Opnd  total_Op  total_Opnd  branchCount  defects 
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64        Float64  Float64  Int64   Int64      Int64    Int64              Float64  Float64    Float64   Float64     Float64      Bool    
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │   122.0     28.0     13.0     12.0    427.0  2546.32     0.03    39.59    56.48      1.02259e5     0.85  5681.04      86          7       18                  0     25.0       46.0     275.0       150.0         55.0     true
   2 │    65.0      2.0      1.0      2.0    167.0  1008.

In [53]:

#dataFrame转换为Array
train=Array(train)
test=Array(test)
function bool(x)
    if x<0.5
        return false
    else
        return true
    end
    
    
end
model=build_forest(train[:,end],train[:,1:end-1],2,20,0.5,8)
trainpredictions = apply_forest(model, train[:,1:end-1])
testpredictions = apply_forest(model, test[:,1:end-1])


trainpredictions .=   map(x -> bool(x), trainpredictions)
testpredictions .=   map(x -> bool(x), testpredictions)
println(trainpredictions[1:20])
println(testpredictions[1:20])


trainaccuracy = sum(trainpredictions .== train[:,end]) / length(train[:,end])
testaccuracy = sum(testpredictions .== test[:,end]) / length(test[:,end])

println("测试集准确率：",testaccuracy)
println("训练集准确率：",trainaccuracy)


[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
测试集准确率：0.7189464556687621
训练集准确率：0.7330496992032952


In [54]:
submission=CSV.read("test.csv",DataFrame)
submission=submission[:,2:end]
submission=Array(submission)
submissionpredictions = apply_forest(model, submission[:,:])
submissionpredictions .=   map(x -> bool(round(Int,x)), submissionpredictions)




67842-element Vector{Float64}:
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 1.0
 0.0
 ⋮
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0

In [55]:
df_predictions = DataFrame(defects=submissionpredictions)

df=CSV.read("sample_submission.csv",DataFrame)
df[:,2]=df_predictions[:,1]

CSV.write("sample_submission.csv",df)

"sample_submission.csv"